## 🚀 大语言模型微调实战教程

### 📘 欢迎来到大语言模型微调之旅！

在这个教程中，我们将一起探索如何将强大的预训练模型转变为您专属的AI助手。无论您是AI初学者还是经验丰富的开发者，这份教程都将以最直观的方式带您掌握模型微调的核心技术。

---

### 🎯 为什么需要模型微调？

想象一下，预训练的大语言模型就像一位博学的通才，虽然知识渊博，但可能不太了解您的特定业务需求。而**模型微调（Fine-tuning）**就是将这位"通才"培养成您领域的"专家"的过程。

#### 核心概念解析：
- **预训练模型**：在海量文本上训练的基础模型，拥有广泛的语言理解能力
- **微调过程**：使用您的领域数据，让模型学习特定任务的模式和知识
- **最终效果**：获得一个既保留通用能力，又精通特定任务的定制化模型

---

### 🛠️ 本教程的技术选型

我们精心选择了一套**高效、易用、资源友好**的技术栈：

#### 1. **Unsloth 加速引擎** ⚡
- 提供 **2倍训练加速**
- 减少 **60%显存占用**
- 让您在普通GPU上也能训练大模型！

#### 2. **LoRA 微调技术** 🎯
- 只训练 **0.1%的参数**，却能达到接近全量微调的效果
- 训练后的适配器文件仅几十MB，易于部署和分享
- 支持多个LoRA适配器灵活切换

#### 3. **4-bit 量化技术** 💾
- 将模型大小压缩至原来的 **1/4**
- 在保持性能的同时，大幅降低硬件要求
- 让 24GB 显存的消费级显卡也能运行 70B 级别的模型

#### 4. **基座模型：百度文心 ERNIE-4.5** 🤖
我们选择了百度最新的 ERNIE-4.5 系列模型作为基座：
- **0.3B 版本**：适合快速实验和学习（本教程默认）
- **21B 版本**：适合生产环境的高性能需求

---

### 📚 学习路线图

```
第一步：环境搭建（10分钟）
   ↓
第二步：模型加载与配置（5分钟）
   ↓
第三步：数据准备与格式化（10分钟）
   ↓
第四步：启动训练（20分钟）
   ↓
第五步：保存与部署（5分钟）
```

让我们开始这段激动人心的AI之旅吧！🎉

## 📦 第一步：环境准备

### 🔧 安装必要的库

首先，我们需要安装几个关键的Python库。这些库将为我们提供模型训练所需的所有功能。

**提示**：安装过程大约需要2-3分钟，可以趁此时间阅读后续的内容说明。

In [ ]:
# 从 GitHub 的源代码安装最新版本 unsloth
# 这样可以确保我们使用的是最新功能和修复。
!pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo
!pip install -U transformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-2ngag4f7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-2ngag4f7
  Resolved https://github.com/unslothai/unsloth.git to commit 9c8735ae4af8d4cbd26d12d02e2c294b8227579f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.8.7-py3-none-any.whl size=309679 sha256=d6a5c6f9ed2faeaacc3316a767b611f19c052f8b590ad10f0b59aa7392fdf90e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2x_tdr_f/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.3 MB/s eta 0:00:00
   ━━━

## 🤖 第二步：加载模型并配置 LoRA

### 💡 理解 LoRA：让大模型微调变得简单

在开始代码之前，让我们先理解一下 **LoRA（低秩适配）** 的魔力：

#### 传统微调 vs LoRA 微调

| 对比项 | 传统全量微调 | LoRA 微调 |
|--------|------------|-----------|
| 训练参数量 | 100% (数十亿) | 0.1-1% (数百万) |
| 显存需求 | 极高 (>80GB) | 较低 (8-24GB) |
| 训练时间 | 数天 | 数小时 |
| 模型文件大小 | 完整模型大小 | 仅适配器几十MB |
| 效果 | 最佳 | 接近最佳 |

#### LoRA 的工作原理

想象模型的知识更新就像给房子装修：
- **传统方法**：拆掉重建整个房子（训练所有参数）
- **LoRA方法**：只在关键位置加装"适配器"（训练少量新参数）

具体来说，LoRA 在模型的注意力层旁边添加两个小矩阵（A和B），通过训练这两个小矩阵来调整模型行为：
```
原始输出 = W × 输入
LoRA输出 = W × 输入 + (B × A) × 输入
           ↑原始不变    ↑新增的可训练部分
```

#### 关键参数解释

- **r (rank)**：LoRA矩阵的秩，控制适配器的容量
  - r=8：轻量级微调，适合简单任务
  - r=16：平衡选择（本教程使用）
  - r=32+：复杂任务，需要更多新知识
  
- **lora_alpha**：缩放因子，通常设为 r 的 2倍
- **lora_dropout**：防止过拟合，通常设为 0.05-0.1
- **target_modules**：要添加LoRA的模块
  - "all-linear"：所有线性层（推荐）
  - 也可指定特定层如 ["q_proj", "v_proj"]

In [ ]:
# 1) 加载模型
from unsloth import FastModel
import torch

MAX_LEN = 4096

# 有两个大小的模型可供下面的流程运行使用：baidu/ERNIE-4.5-0.3B-PT, baidu/ERNIE-4.5-21B-A3B-PT
# 如果是第一次快速跑脚本推荐用0.3B小模型进行体验（但大多实际下游任务可能比较难以学会）
# 如果是正常任务训练推荐切换到A100显卡上基于21B模型进行训练。
model, tokenizer = FastModel.from_pretrained(
    model_name      = "baidu/ERNIE-4.5-0.3B-PT", # baidu/ERNIE-4.5-0.3B-PT, baidu/ERNIE-4.5-21B-A3B-PT
    # max_seq_length  = MAX_LEN,
    # load_in_4bit    = False,            # QLoRA 常用配置，如果显存吃紧推荐设置为True
    # load_in_8bit    = False,
    # full_finetuning = False,           # LoRA/QLoRA 训练；要全参训练可设 True（需更大显存）
    trust_remote_code = True,          # ERNIE 需要
)

# 2) 配置 LoRA
from unsloth import FastLanguageModel
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, lora_alpha = 32, lora_dropout = 0.05,
    # 这里的 r 可以根据任务难度和数据量调大（更大的rank可以存储更多的新知识），alpha推荐设置为rank的 2 倍
    target_modules = "all-linear",     # Unsloth 会做自动打点；也可指定具体模块
    use_rslora = True,                 # 常用的稳定配置
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.8.7: Fast Ernie4_5 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Ernie4_5 does not support SDPA - switching to eager!


model.safetensors:   0%|          | 0.00/722M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

chat_template.jinja:   0%|          | 0.00/754 [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


## 📊 第三步：准备训练数据

### 📝 数据格式化的重要性

数据是模型微调的"教材"。就像教学需要合适的教材一样，模型训练也需要正确格式的数据。

#### 为什么数据格式如此重要？

1. **模型理解**：模型在预训练时已经学会了特定的对话格式
2. **性能影响**：错误的格式会导致模型"听不懂"指令
3. **一致性**：保持格式一致能让模型更好地泛化

#### ERNIE 模型的对话格式

ERNIE-4.5 使用特定的对话模板：
```
<|begin_of_sentence|>User: [用户输入]
Assistant: [助手回复]<|end_of_sentence|>
```

这些特殊标记帮助模型区分：
- 对话的开始和结束
- 用户输入和AI回复的界限
- 多轮对话的上下文

#### 数据集选择建议

本教程使用 Microsoft 的 Orca Math 数据集作为示例，但您可以根据需求选择：

- **通用对话**：ShareGPT、Alpaca 等
- **数学推理**：GSM8K、MATH 等
- **代码生成**：CodeAlpaca、StarCoder 等
- **领域特定**：医疗、法律、金融等专业数据集
- **自定义数据**：您自己的业务数据（推荐）

In [ ]:

# 3) 准备数据（示例：ShareGPT / chat 格式）
from datasets import load_dataset
ds = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:1%]")  # 替换为你的数据

def format_chat(ex):
    msgs = [{"role": "user", "content": ex["question"]}]
    if "answer" in ex and ex["answer"]:
        msgs.append({"role": "assistant","content": ex["answer"]})
    return {"text": tokenizer.apply_chat_template(
        msgs, tokenize=False, add_generation_prompt=False
    )}
ds = ds.map(format_chat, remove_columns=ds.column_names)

# 打印一条数据看看给模型的底层输入序列
ds[:1]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': ['<|begin_of_sentence|>User: Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.\nAssistant: If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|end_of_sentence|>']}

### 🔍 数据格式验证

从上面的输出可以看到，数据已经被正确格式化为 ERNIE 模型期望的格式：
- `<|begin_of_sentence|>` 标记对话开始
- `User:` 和 `Assistant:` 明确区分角色
- `<|end_of_sentence|>` 标记对话结束

这种格式确保模型能够正确理解任务边界和角色切换。

## 🚂 第四步：配置并启动训练

### ⚙️ 训练参数详解

训练配置是微调成功的关键。让我们详细了解每个参数的作用和调优建议：

#### 🎯 批处理与梯度累积

**有效批大小 = per_device_train_batch_size × gradient_accumulation_steps × GPU数量**

在我们的配置中：
- `per_device_train_batch_size = 1`：每次前向传播处理1个样本
- `gradient_accumulation_steps = 8`：累积8次梯度后更新
- **有效批大小 = 1 × 8 = 8**

💡 **调优建议**：
- 显存充足：增大 `per_device_train_batch_size`
- 显存不足：减小批大小，增加梯度累积步数
- 目标：有效批大小通常在 8-32 之间效果较好

#### 📈 学习率与优化器

- `learning_rate = 1e-4`：LoRA 微调的典型学习率
  - 太高（>5e-4）：训练不稳定，损失震荡
  - 太低（<1e-5）：收敛缓慢，效果不佳
  - 经验值：LoRA 用 1e-4 到 2e-4，全量微调用 1e-5 到 5e-5

- `optim = "adamw_8bit"`：8-bit AdamW 优化器
  - 节省 75% 优化器状态内存
  - 性能损失几乎可以忽略
  - 适合大模型和长序列训练

- `lr_scheduler_type = "linear"`：线性学习率衰减
  - 训练初期学习率高，快速学习
  - 训练后期学习率低，精细调整
  - 其他选择：cosine（余弦衰减）、constant（恒定）

#### 🔄 训练轮数与步数

- `num_train_epochs = 1`：训练轮数
  - 小数据集（<10k）：3-5 轮
  - 中等数据集（10k-100k）：1-3 轮
  - 大数据集（>100k）：1 轮或按步数

- 也可以使用 `max_steps` 替代：
  - 快速测试：50-100 步
  - 正常训练：500-2000 步
  - 深度训练：5000+ 步

#### 💾 检查点保存

- `save_strategy = "steps"`：按步数保存
- `save_steps = 200`：每200步保存一次
  - 便于中断后恢复训练
  - 可以选择最佳检查点
  - 建议：设为总步数的 10-20%

#### 🚀 性能优化

- `fp16 = True`：使用半精度训练
  - 速度提升 2x
  - 显存减少 50%
  - 适合大多数 GPU（V100、T4、RTX 等）

- `bf16 = False`：BFloat16 精度
  - 更好的数值稳定性
  - 需要 A100、H100 等新GPU
  - 如果支持，优先使用 bf16

- `packing = True`：样本打包
  - 将短样本拼接，提高GPU利用率
  - 特别适合长度差异大的数据集
  - 可提升 2-5x 训练速度

#### 📊 监控与调试

- `logging_steps = 5`：每5步记录一次
  - 观察损失下降趋势
  - 及时发现训练问题
  - 生产环境可设为 10-50

- `report_to = "none"`：不上报到外部平台
  - 可选："tensorboard"、"wandb"
  - 便于可视化和团队协作

### 🎓 训练过程解读

启动训练后，您会看到类似这样的输出：
```
Step | Training Loss | Validation Loss
-----|---------------|----------------
10   | 2.345         | -
20   | 1.876         | -
30   | 1.234         | -
```

**正常情况**：
- Loss 持续下降（从 2-3 降到 0.5-1.5）
- 下降速度逐渐放缓
- 没有突然的跳变

**异常情况及解决**：
- Loss 不下降 → 检查数据格式，增大学习率
- Loss 震荡 → 减小学习率，增加梯度累积
- Loss 突增 → 可能过拟合，添加 dropout，减少训练步数

In [ ]:
# 4) 用 TRL 的 SFTTrainer 训练（Unsloth 官方推荐方式）
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    learning_rate = 1e-4,             # 长训可降到 2e-5
    logging_steps = 5,
    num_train_epochs = 1,             # 或用 max_steps
    fp16 = True,
    bf16 = False,
    optim = "adamw_8bit",
    lr_scheduler_type = "linear",
    weight_decay = 0.01,
    output_dir = "outputs",
    save_strategy = "steps",
    save_steps = 200,                 # 200步保存一次训练检查点，便于resume
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    dataset_text_field = "text",
    args = args,
    max_seq_length = MAX_LEN,
    packing = True,                   # 让样本拼接更高效
    use_cache=False,
)

trainer_stats = trainer.train()       # 可传 resume_from_checkpoint=True 续训


## 💾 第五步：保存微调后的模型

In [ ]:
model.save_pretrained_merged(
    "ernie-4.5-0.3b-sft-merged",
    tokenizer,
    save_method = "merged_16bit",
)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 1 files from cache to ernie-4.5-0.3b-sft-merged.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:02<00:00,  2.55s/it]


保存的训练后权重在：/content/ernie-4.5-0.3b-sft-merged

In [ ]:
# 如果需要，可以清理内存
# import gc
# del model, tokenizer, trainer, dpo_trainer
# gc.collect()
# torch.cuda.empty_cache()